In [ ]:
import os
import nltk
import csv
from matplotlib.pyplot import cm
import numpy as np
from nltk.corpus import stopwords
import math

import matplotlib.pyplot as plt


from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from wordcloud import WordCloud

from bs4 import BeautifulSoup
from community import community_louvain
from fa2 import ForceAtlas2

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [ ]:
# To achieve default 
average_length_of_sentence = 114 # Set this to 79
sentences_for_con = 3 # And this to 1

# Sentiment Analysis
>We want to be able to analyse the sentiment of a character throughout the books. Our idea for achieving this is to use concordance from nltk with the character name, this way we will get all of the context surrounding a character. For each of these occurrences we can compute the sentiment for the context, and use that sentiment as a representative for the character. 

## Sentiment calculations from LabMT1.0 vs VADER-Sentiment
>We have considered two options for calculating our sentiment. Either we could use the the LabMT1.0 data set to find the sentiment of a portion of text by assigning each word in that text a value based on LabMT1.0, and then taking the average of those words. Or we could use <a href="https://github.com/cjhutto/vaderSentiment/blob/master/README.rst">VADER-Sentiment</a>. We wanted to experiment with the VADER solution, since our initial findings for sentiment using LabMT1.0 had very similar values around 5.5. To experiment we have made a graph of sentiments for each chapter of Book 7, for Harry Potter, Voldemort, and Snape for both methods:

###  Sentiment for concordance of character, LabMT1.0
> First we create a list of tuples containing each word and the average happiness for that word. This allows us to go through a portion of text and look up the average happiness for each word.

In [ ]:
# Create an empty list to store tuples of words and their 
# average happiness score
sent_list_labmt10 = []

# Read in the .tsv file
with open("LabMT1.0.tsv", "r", encoding="utf8") as sent_file:
    tsv_reader = csv.DictReader(sent_file, delimiter="\t")
    # For each sentiment in the file, save the word and average happiness in a tuple
    # and add it to the list
    for sent in tsv_reader:
        word = sent["word"]
        average = sent["happiness_average"]
        sent_list_labmt10.append((word, average))

>We now define a function to calculate the average sentiment for a set of tokens:

In [ ]:
# Defining function to calculate sentiment for a list of tokens
# Function for calculating the sentiment of a file from the frequency distribution for that file
def sentiment_labmt10(tokens):
    # Total sentiment score of file
    sent_sum_labmt10 = 0.0
    # Total number of occurences of words
    occ_sum_labmt10 = 0
    
    # For each token and associated number of occurences
    for token, occ in tokens.items():
        # If the token is in the given list of words with rated happiness
        for word, score in sent_list_labmt10:
            if token == word:
                sent_sum_labmt10 += (float(score) * occ)
                occ_sum_labmt10 += occ
    return sent_sum_labmt10 / occ_sum_labmt10


>And a function to calculate the sentiment for a concordance list found by nltk, and define a list of stopwords to be filtered out:

In [ ]:
stopwords_labmt10 = nltk.corpus.stopwords.words('english')

In [ ]:
# Finding sentiment from a concordance list
def con_sentiment_labmt10(con_list):
    sent_sum = 0
    line_num = 0
    for item in con_list:
        left = [ch.lower() for ch in item.left if ch.isalpha()]
        right = [ch.lower() for ch in item.right if ch.isalpha()]
        # Make left and right into one list and remove stopwords
        combined = [w for w in (left + right) if w not in stopwords_labmt10]
        
        # Make frequency distribution 
        fd = nltk.FreqDist(combined)
        sent_sum += sentiment_labmt10(fd)
        line_num += 1
    if line_num == 0:
        return None # Character had no appearences in chapter
    return sent_sum / line_num

>We then run our test as described previously:

In [ ]:
# Define character list with the three characteres
char_list_labmt10 = ["Harry_Potter", "Severus_Snape", "Tom_Riddle"] 

# Create a list of the chapters of book 7
chapters_labmt10 = os.listdir("B7")

# Init a list to tuples (chraracter, [sentiments for each chapter for that character])
sentiment_by_character_labmt10 = []

# For each of the characters
for character in char_list_labmt10:
    sentiments = []
    # For each chapter
    for chapter in chapters_labmt10:
        # Read in and tokenize the chapter
        if "replaced" in chapter:
            with open("B7/" + chapter) as f:
                    raw = f.read()
            tokens = nltk.word_tokenize(raw)
            text = nltk.Text(tokens)
        
            # Make concordance for that character
            con = text.concordance_list(character, width = sentences_for_con * average_length_of_sentence)
        
            # Calculate sentiments and append to the list for that character
            sentiments.append(con_sentiment_labmt10(con))
    # Append the character with its full sentiment list
    sentiment_by_character_labmt10.append((character, sentiments))

# https://stackoverflow.com/questions/4971269/how-to-pick-a-new-color-for-each-plotted-line-within-a-figure-in-matplotlib
color = iter(cm.rainbow(np.linspace(0, 1, len(char_list_labmt10))))

for name, sentiments in sentiment_by_character_labmt10:
    c = next(color)
    plt.plot(sentiments, c=c, label=name)

plt.legend()
plt.xlabel('Chapter')
plt.ylabel('Average sentiment')
plt.title('Average sentiment by chapter for selected characters')
plt.figtext(.5, -0.1, f"Plot of the average sentiment for Harry, Snape, and Voldemort in book 7 when calculating sentiment from LabMT1.0.", ha="center")

plt_labmt = plt

### Sentiment for concordance of character, vaderSentiment
>For VADER we use the same approach and code, but we have to redefine how we calculate sentiment from concordance, since we just have to pass a string to the analyzer:

In [ ]:
# Finding sentiment from a concordance list
def con_sentiment(con):
    sent_sum = 0
    line_num = 0
    combined = con.left + con.right
    combined = " ".join(combined)

    vs = analyzer.polarity_scores(combined)

    if vs == 0:
        return None # Character had no appearences in chapter
    return vs["compound"]

In [ ]:
# Define character list with the three characteres
char_list_vader = ["Harry_Potter", "Severus_Snape", "Tom_Riddle"] 

# Create a list of the chapters of book 7
chapters_vader = os.listdir("B7")

# Init a list to tuples (chraracter, [sentiments for each chapter for that character])
sentiment_by_character_vader  = []

# For each of the characters
for character in char_list_vader :
    sentiments = []
    for chapter in chapters_vader :
        if "replaced" in chapter:
            with open("B7/" + chapter) as f:
                    raw = f.read()
            tokens = nltk.word_tokenize(raw)
            text = nltk.Text(tokens)
            cons = text.concordance_list(character, width = sentences_for_con * average_length_of_sentence)
            sent_sum = 0
            lines = 0
            for con in cons:
                sent_sum += con_sentiment(con)
                lines += 1
            if lines == 0:
                sentiments.append(None)
            else: 
                sentiments.append(sent_sum / lines)
    sentiment_by_character_vader .append((character, sentiments))

# https://stackoverflow.com/questions/4971269/how-to-pick-a-new-color-for-each-plotted-line-within-a-figure-in-matplotlib
color_vader  = iter(cm.rainbow(np.linspace(0, 1, len(char_list_vader ))))

for name, sentiments in sentiment_by_character_vader :
    c = next(color_vader)
    plt.plot(sentiments, c=c, label=name)

plt.axhline(y = 0.05, color ="purple", linestyle = '--', label="Neutral region")
plt.axhline(y = -0.05, color ="purple", linestyle = '--')

plt.legend()
plt.xlabel('Chapter')
plt.ylabel('Average sentiment')
plt.title('Average sentiment by chapter for selected characters')
plt.figtext(.5, -0.1, f"Plot of the average sentiment for Harry, Snape, and Voldemort in book 7 when calculating sentiment with VADER.", ha="center")

plt_vader = plt

### Conclusion: LabMT1.0 vs vaderSentiment
>Both results are similar, but we see an advantage in using VADER when we consider the sentiment for Harry Potter. Looking at the graphs we can see that there are similar trends for Harry throughout the book, but with VADER the sentiment becomes more consistent, in that it appears to be in the neutral region at some points, and then go out of it. On the other hand the MatLab1.0 seems to indicate that Harry is well above 5.1 throughout the book, which we would consider to be above neutral. Based on this preliminary test we believe that we will get a more clear picture from VADER.

## Functions for calculating sentiment
>The following section contains various functions we have defined to find and plot sentiment for various cases.

### ```sent_chars_book(char_list, path_to_book)```
>The function takes a list of character names and a path to a book. It computes the average sentiment of each character in the list throughout the book on a chapter basis. This can be used to find out how the sentiment of a single character changes throughout a book or a group of characters such as a house.

In [ ]:
"""
Input: A list of character names to look for as tokens
        and a path to the chapters of the book
Output: A list of tuples, with the character name and a list
        of the sentiments for each chapter for that character
"""
def sent_chars_book_list(char_list, book_list, sentences):
    # Init list to hold tuples
    sentiment_of_character = [(n, []) for n in char_list]
    #print(sentiment_of_character)
    #print(len(sentiment_of_character))
    #print(char_list)
    
    chapter_counter = 0
    for book in book_list:
        
        # For each chapter
        for chapter in os.listdir(book):
            if "replaced" in chapter:
                # Read in the chapter and tokenize
                with open(book + chapter) as f:
                    raw = f.read()
                tokens = nltk.word_tokenize(raw)
                text = nltk.Text(tokens)

                character_counter = 0
                # For each character in the given list
                for character in char_list:
                    # Make concordance for that character in that chapter
                    cons = text.concordance_list(character, width = sentences * average_length_of_sentence)
                    #print(character)
                    #print(f"character = {character}, sentiment_of_character[{character_counter}] = {sentiment_of_character[character_counter]}")

                    sent_sum = 0
                    lines = 0
                    # For each concordance line
                    for con in cons:
                        # Calculate the sentiment for that concordance line
                        sent_sum += con_sentiment(con)
                        lines += 1
                    if lines == 0:
                        # If there were no lines, the character did not appear
                        sentiment_of_character[character_counter][1].append(None)
                    else: 
                        sentiment_of_character[character_counter][1].append(sent_sum / lines)
                    if character_counter > len(sentiment_of_character):
                        print("!!!!!!!!!!!!!!!!!!!!!!!! WRONG")
                    character_counter += 1
    # Returns a list of the sentiments for that character for each chapter of that book
    return sentiment_of_character

### ```sent_book(path_to_book)```
>Calculates the sentiment for a book on chapter basis. In this function each chapter of a book is read in and the sentiment for the text is calculated. This allows us to see how the sentiment for a book changes as it progresses.

In [ ]:
"""
Input: The path to a book.
Output: A list of sentiments for each chapter of the book.
"""
def sent_book(path_to_book):
    chapters = os.listdir(path_to_book)

    # Making a data table (char_list)x(num_chapters) to hold sentiments for each char in each chap
    # tuple list with tuples (character, [sent chapter1, sent chapter2, ...])
    sentiments_by_chapter = []
    
    for chapter in chapters:
        if "replaced" in chapter:
            with open(path_to_book + chapter) as f:
                raw = f.read()
            sentiments_by_chapter.append(analyzer.polarity_scores(raw)["compound"])
    # Returns a list of the sentiments for that character for each chapter of that book
    return sentiments_by_chapter

### ```sent_group(group, label, book_list)```
>This function calculates the average sentiments for all of the names given in ```group``` and returns it as as the sentiment for the name from ```label_group```. The sentiments are calculated from the books given in ```book_list```. e.g. given the list of names of Gryffindor students, with label "Gryffindor" returns a tuple ("Gryffindor", sentiment list), where the sentiment list contains the average sentiment for each chapter for those students.

In [ ]:
def avgsent_group(group, label, book_list, sentences):
    # Count number of chapters
    num_chapters = 0
    for book in book_list:
        for chapter in os.listdir(book):
            if "replaced" in chapter:
                num_chapters += 1
    #print(f"num_chapters = {num_chapters}")
    sent_chapters = [0] * num_chapters
    sent_group = (label, sent_chapters)
    
    #print(f"length of sent_chapters = {len(sent_chapters)}")
    #print(f"length of sent_group[1] = {len(sent_group[1])}")
    
    # Counter for current chapter
    c = 0
    
    # For each book in the list
    for book in book_list:  
        # For each chapter going by numbering
        for chapter in os.listdir(book):
            if "replaced" in chapter:
                # Init counter for counting occurences in chapter for average
                occurences = 0

                # Read in the chapter
                with open(book + chapter) as f:
                        raw = f.read()
                tokens = nltk.word_tokenize(raw)
                # Prepare nltk text
                text = nltk.Text(tokens)

                # For each member of the group
                for member in group:
                    # Make concordance for that member for that chapter
                    con_list = text.concordance_list(member, width = sentences * average_length_of_sentence)

                    # For each concordance line in the list
                    for con in con_list:
                        # Calculate the sentiment
                        sent = con_sentiment(con)
                        if sent != 0:
                            # Sum up the sentiment for that chapter for that member
                            # with sentiments for all other members of group
                            #print(f"c = {c}")
                            sent_group[1][c] += sent
                            occurences += 1
                # Divide by the total number of occurences 
                if occurences == 0:
                    sent_group[1][c] = None
                else:
                    sent_group[1][c] = sent_group[1][c] / occurences
                c += 1
    return sent_group                    

### ```plot_sentiments(sentiment_by_character, figure_text, xs_vertical_lines, show_labels)```
>The functions takes a list of tuples, where each tuple contains a name and a list of sentiments. A figure text, a list of tuples for placing vertical lines with labels, and a ```True```/```False```flag for show labels. This function may take the result of ```sent_char_books``` a sinput for the list of tuples with names and sentiments. This allows for fast and simple plotting.

In [ ]:
"""
Input: A list of tuples containing a name and a sentiment list, a figure text, 
       a list of tuples with labels and coordinates for vertical lines, and a
       true/false value for showing labels.
Output: void, shows a plot
"""
def plot_sentiments(sentiment_by_character, figure_text, xs_vertical_lines, show_legend):
    # Init iterator
    color = iter(cm.rainbow(np.linspace(0, 3, len(sentiment_by_character)*3)))
    plt.xlim(0, len(sentiment_by_character[0][1]))
    
    for name, sentiments in sentiment_by_character:
        # Try to give colors according the name associated with the sentiments
        if ("Gryffindor" in name):
            c = "#a6332e"
            a = 1
        elif ("Slytherin" in name and name != "Salazar_Slytherin"):
            c = "#366447"
            a = 1
        elif ("Hufflepuff" in name):
            c = "#efbc2f"
            a = 1
        elif ("Ravenclaw" in name):
            c = "#3c4e91"
            a = 1
        elif ("Average" in name):
            c = "black"
            a = 0.75
        else:
            c = next(color)
            a = 0.75
        plt.plot(sentiments, c=c, alpha=a, label=name, linewidth=2)
    
    # Make horizontal lines to indicate the neutral region
    plt.axhline(y = 0.05, color ="black", linestyle = '-', label="Neutral", alpha=0.5)
    plt.axhline(y = -0.05, color ="black", linestyle = '-', alpha=0.5)
    
    # If vertical lines for book has been specified insert them
    for book, label, vertical_line in xs_vertical_lines:
        plt.axvline(x = vertical_line, color = 'black')
        
    # Get list of labels and vertical_lines
    if xs_vertical_lines != []:
        xs = []
        labels = []
        for book, x_coordinate, label in xs_vertical_lines:
            xs.append(x_coordinate)
            labels.append(label)
        plt.xticks(labels, xs ,rotation=45, fontsize=16)

    # If legend has been requested
    if show_legend:
        plt.legend()

    #plt.xlabel('Chapter')
    plt.ylabel('Average sentiment', fontsize=16)
    plt.figtext(.5, -0.35, figure_text, ha="center")

    plt.show()

## Sentiment for books
>First we want to explore how the sentiment is throughout the books. We have two ideas for measuring this: Take each chapter as a text and have VADER analyze the sentiment of that text for us. Or for each character from our list of characters, make concordance for each chapter for that character, and divide it by the total number of concordance lines for that chapter. We are going to test these two methods out to see which is more expressive:

>We start by defining some list needed for plotting and reading in all the books:

In [ ]:
# Tuple list containing the first chapter of the next book, e.g. ("B1", 18), indicates
# all chapters up to 18 excluded are from book 1
book_list_wchapter = [("B1", "Philosopher's Stone", 0), 
                      ("B2", "Chamber of Secrets", 18),
                      ("B3", "Prisoner of Azkaban", 35), 
                      ("B4", "Goblet of Fire", 57), 
                      ("B5", "Order of the Phoenix", 94), 
                      ("B6", "Half-Blood Prince", 134), 
                      ("B7", "Deathly Hallows", 162)]
# Lists of paths to the folders holding the text from the chapters
book_list = ["B1/", "B2/", "B3/", "B4/", "B5/", "B6/", "B7/"]

In [ ]:
# Init list to hold sentiment values
series_sent = []
# Compute and append the sentiment values for the seven books
series_sent += sent_book("B1/")
series_sent += sent_book("B2/")
series_sent += sent_book("B3/")
series_sent += sent_book("B4/")
series_sent += sent_book("B5/")
series_sent += sent_book("B6/")
series_sent += sent_book("B7/")

In [ ]:
# Figure text
sent_by_chapter = "Sentiment by chapter for entire series analyzed one chapter at a time"
# Adjust for a wider figure size
plt.rcParams['figure.figsize'] = [15, 5]
# Plot the sentiment values for all of the books when VADER analyzed each chapter as a whole text
plot_sentiments([("Sentiment for series", series_sent)], sent_by_chapter, book_list_wchapter, False)

>The above figure does not convey changes in the book very well, at most it seems that we can get an idea of the overall tone of the chapter, but not how it relates to the other chapters or how the story evolves and changes. 

>For the next part we are going to try the approach with making concordance for all of the characters for each chapter and taking the average. However ```concordance_list``` uses a default width of 80 characters around the string that it is making concordance for. First we have to investigate if this default value is good. To investigate this we first determine the average length in characters for the books:

In [ ]:
#nltk.download('punkt')

In [ ]:
# What the average length of a sentence in harry potter? in characters
# Read in all of the chapters and count the length of the sentences, divide by the number ound
# use this for average to determine how many sentences we want included for each concordance?
number_of_sentences = 0
number_of_characters = 0
for book in book_list:
    for chapter in os.listdir(book):
        if "replaced" in chapter:
            with open(book + chapter) as f:
                raw = f.read()
            sentences = nltk.sent_tokenize(raw)
            for sentence in sentences:
                number_of_characters += len(sentence)
                number_of_sentences += 1
average_characters_in_sent = number_of_characters / number_of_sentences
print(average_characters_in_sent)

>The average sentence in the books is 114 characters long. Our default width for a concordance should then be 114 characters, assuming the character name is in the middle of the sentence and has an even length. So now we can test out how many sentences should be used. We are going to test with a smaller size of a half sentence, default, one sentence, two, three, and four to start with:

In [ ]:
con_sentences_test = []
average_length_of_sentence = 79
con_sentences_test.append(avgsent_group(["Harry_Potter"], "Harry Potter Default", ["B1/"], 1))
average_length_of_sentence = 114
con_sentences_test.append(avgsent_group(["Harry_Potter"], "Harry Potter 1", ["B1/"], 1))
con_sentences_test.append(avgsent_group(["Harry_Potter"], "Harry Potter 2", ["B1/"], 2))
con_sentences_test.append(avgsent_group(["Harry_Potter"], "Harry Potter 3", ["B1/"], 3))
con_sentences_test.append(avgsent_group(["Harry_Potter"], "Harry Potter 4", ["B1/"], 4))
plot_sentiments(con_sentences_test, "Average sentiment for Harry Potter in book one, with 0.5 sentences used for concordance", [], True)

In [ ]:
con_sentences_test = []
con_sentences_test.append(avgsent_group(["Harry_Potter"], "Harry Potter 8", ["B1/"], 8))
con_sentences_test.append(avgsent_group(["Harry_Potter"], "Harry Potter 16", ["B1/"], 16))
con_sentences_test.append(avgsent_group(["Harry_Potter"], "Harry Potter 32", ["B1/"], 32))
con_sentences_test.append(avgsent_group(["Harry_Potter"], "Harry Potter 64", ["B1/"], 64))
plot_sentiments(con_sentences_test, "Average sentiment for Harry Potter in book one, with differening number of sentences used for concordance", [], True)

In [ ]:
# Create character tuple list
characters = []

# Read the character with their attributes from our .csv file
with open("HP_characters.csv", "r", encoding="utf8") as sent_file:
    csv_reader_characters = csv.reader(sent_file, delimiter=",")
    for row in csv_reader_characters:
        name = row[0].replace(' ', '_')
        parentage = row[1]
        house = row[2]
        occupation = row[3]
        characters.append((name, parentage, house, occupation))

In [ ]:
# Extract the character names from our list with attributes
character_names = [n for n, b, h, o in characters]

In [ ]:
con_test_all = []
con_test_all.append(avgsent_group(character_names, "All characters, 2 sentences con", ["B1/"], 2))
con_test_all.append(avgsent_group(character_names, "All characters, 3 sentences con", ["B1/"], 3))
con_test_all.append(avgsent_group(character_names, "All characters, 4 sentences con", ["B1/"], 4))
con_test_all.append(avgsent_group(character_names, "All characters, 32 sentences con", ["B1/"], 32))
con_test_all.append(("book 1 sent", sent_book("B1/")))

In [ ]:
plot_sentiments(con_test_all, sent_by_chapter, [], True)

In [ ]:
# Calculate the average sentiment for each chapter
avgsent_all = avgsent_group(character_names, "Average sentiment of all characters", book_list, sentences_for_con)

In [ ]:
sentiment_books = "Average sentiment from concordance of character names throughout the books."
plot_sentiments([avgsent_all], sentiment_books, book_list_wchapter, True)

## Sentiment for houses

In [ ]:
gryffindors = [n for n, p, h, o in characters if h == "Gryffindor"]
hufflepuffs = [n for n, p, h, o in characters if h == "Hufflepuff"]
ravenclaws = [n for n, p, h, o in characters if h == "Ravenclaw"]
slytherins = [n for n, p, h, o in characters if h == "Slytherin"]

print(f"Number of characters from Gryffindor: {len(gryffindors)}")
print(f"Number of characters from Hufflepuff: {len(hufflepuffs)}")
print(f"Number of characters from Ravenclaw: {len(ravenclaws)}")
print(f"Number of characters from slytherin: {len(slytherins)}")

In [ ]:
avgsent_gryffindor = avgsent_group(gryffindors, "Gryffindors", book_list, sentences_for_con)

In [ ]:
avgsent_gryffindor = avgsent_group(gryffindors, "Gryffindors", book_list, sentences_for_con)
avgsent_hufflepuff = avgsent_group(hufflepuffs, "Hufflepuffs", book_list, sentences_for_con)
avgsent_ravenclaw = avgsent_group(ravenclaws, "Ravencalws", book_list, sentences_for_con)
avgsent_slytherin = avgsent_group(slytherins, "Slytherins", book_list, sentences_for_con)

In [ ]:
avg_sent_list = [avgsent_gryffindor, 
                 avgsent_slytherin,
                 avgsent_hufflepuff,
                 avgsent_ravenclaw]
plot_sentiments(avg_sent_list, "Average sentiment for the four houses throughout the books", book_list_wchapter, True)

In [ ]:
label = "Gryffindors"
gryff_sents = sent_chars_book_list(gryffindors, book_list, sentences_for_con) + [avgsent_gryffindor]

In [ ]:
text_gryff_sent = "Average sentiments for Gryffindor, and sentimens for all Gryffindors"
plot_sentiments(gryff_sents, text_gryff_sent, book_list_wchapter, False)

In [ ]:
# Looking at graph above maybe purple does not follow the trend?
# maybe purple is dumbledore, trying to remove:
tup_dict = dict(gryff_sents)
tup_dict.pop('Albus_Dumbledore')
tup_dict.pop('Gryffindors')
gryff_sent = list(tuple(tup_dict.items()))

# Dumbledore has not been removed?

#gryffindors.remove("Albus_Dumbledore")
gryff_sent = gryff_sent + [avgsent_group(gryffindors, "Gryffindors", book_list, sentences_for_con)]
plot_sentiments(gryff_sents, text_gryff_sent, book_list_wchapter, False)

In [ ]:
label = "Slytherins"
slyth_sents = sent_chars_book_list(slytherins, book_list, sentences_for_con) + [avgsent_slytherin]

text_slyth_sent = "Average sentiments for Slytherin, and sentimens for all Slytherins"
plot_sentiments(slyth_sents, text_slyth_sent, book_list_wchapter, False)

In [ ]:
label = "Hufflepuffs"
huff_sents = sent_chars_book_list(hufflepuffs, book_list, sentences_for_con) + [avgsent_hufflepuff]

# Unexpected plot, hvorfor er alle kapitler fra første bog ikke med? Jf. streg for første bog ovenfor
text_huff_sent = "Average sentiments for Hufflepuff, and sentimens for all Hufflepuffs"
plot_sentiments(huff_sents, text_huff_sent, book_list_wchapter, False)

In [ ]:
label = "Ravenclaws"
rave_sents = sent_chars_book_list(ravenclaws, book_list, sentences_for_con) + [avgsent_ravenclaw]

# Unexpected plot, hvorfor er der ikke plot fra 0 og frem?
text_rave_sent = "Average sentiments for Ravenclaw, and sentimens for all Ravenclaws"
plot_sentiments(rave_sents, text_rave_sent, book_list_wchapter, False)

## Highest and lowest sentiments
>Investigating which characters have the highest and lowest sentiments by summing up their average sentiment values for each chapter they appeared in, and dividing by the number of chapters they appeared in into a sentiment score for that character.

In [ ]:
avgsent_all_individual = sent_chars_book_list(character_names, book_list, sentences_for_con)
character_sent_scores = []
for name, sent_list in avgsent_all_individual:
    sent_score = 0
    sent_sum = 0
    chapter_occurences = 0
    for sent in sent_list:
        if sent != None:
            sent_sum += sent
            lines += 1
    sent_score = sent_sum
    character_sent_scores.append((name, sent_score))

In [ ]:
# https://bobbyhadz.com/blog/python-sort-list-of-tuples-by-second-element
sorted_list = sorted(
    character_sent_scores,
    key=lambda t: t[1]
)
top_names = [n for n, s in sorted_list[-5:]]
top = [(n, l) for n, l in avgsent_all_individual if n in top_names]
bottom_names = [n for n, s in sorted_list[0:5]]
bottom = [(n, l) for n, l in avgsent_all_individual if n in bottom_names]

In [ ]:
top_text = "Sentiment values of 5 highest sentiment, with total average for comparison."
plot_sentiments([avgsent_all] + top, top_text, book_list_wchapter, True)

In [ ]:
bottom_text = "Sentiment values of 5 lowest sentiment, with total average for comparison."
plot_sentiments([avgsent_all] + bottom, bottom_text, book_list_wchapter, True)

In [ ]:
sirius_black = [(n, l) for n, l in avgsent_all_individual if n in ["Sirius_Black"]]
sirius_black_text = "Sentiment for Sirius Black"
plot_sentiments(sirius_black, sirius_black_text, book_list_wchapter, True)

## Are Slytherins "worse" than the rest?

In [ ]:
avgsent_all_woslytherin = avgsent_group([c for c in character_names if c not in slytherins], "Average for rest", book_list, sentences_for_con)

In [ ]:
# Comparing the average of Slytherin with the averages of the books
#avgsent_all, average sentiment of all, including Slytherins
#avgsent_slytherin, average sentiment of Slytherins
sent_slytherin_vs_all =  [avgsent_slytherin] + [avgsent_all_woslytherin]
plot_sentiments(sent_slytherin_vs_all, "Average sentiment for Slytherin vs All, throughout the books", book_list_wchapter, False)

In [ ]:
avgsent_all_wohufflepuff = avgsent_group([c for c in character_names if c not in hufflepuffs], "Average for rest", book_list, sentences_for_con)
sent_hufflepuff_vs_all =  [avgsent_hufflepuff] + [avgsent_all_wohufflepuff]
plot_sentiments(sent_hufflepuff_vs_all, "Average sentiment for Hufflepuff vs All, throughout the books", book_list_wchapter, False)

In [ ]:
avgsent_all_wogryffindor = avgsent_group([c for c in character_names if c not in gryffindors], "Average for rest", book_list, sentences_for_con)
sent_gryffindor_vs_all =  [avgsent_gryffindor] + [avgsent_all_wogryffindor]
plot_sentiments(sent_gryffindor_vs_all, "Average sentiment for Hufflepuff vs All, throughout the books", book_list_wchapter, False)

In [ ]:
avgsent_all_woravenclaw = avgsent_group([c for c in character_names if c not in hufflepuffs], "Average for rest", book_list, sentences_for_con)
sent_ravenclaw_vs_all =  [avgsent_ravenclaw] + [avgsent_all_woravenclaw]
plot_sentiments(sent_ravenclaw_vs_all, "Average sentiment for Hufflepuff vs All, throughout the books", book_list_wchapter, False)

In [ ]:
house_sents_to_plot = [sent_ravenclaw_vs_all] + [sent_hufflepuff_vs_all] + [sent_slytherin_vs_all] + [sent_gryffindor_vs_all]

fig, axs = plt.subplots(4, sharex=True)
fig.set_figheight(20)

#axs[3].set_xticks([])

axs[0].plot(sent_slytherin_vs_all[0][1], c="springgreen", alpha=1, linewidth=2)
axs[0].plot(sent_slytherin_vs_all[1][1], c="black", alpha=1, linewidth=2)
axs[0].axhline(y = 0.05, color ="black", linestyle = '-', label="Neutral", alpha=0.5)
axs[0].axhline(y = -0.05, color ="black", linestyle = '-', alpha=0.5)
axs[0].set_xlim(0, len(sent_ravenclaw_vs_all[0][1]))
axs[1].set_ylim(-1, 1)
axs[0].set_ylabel("Average sentiment", fontsize=16)
axs[0].tick_params(axis='y', which='major', labelsize=16)
axs[0].set_xticks([])

axs[1].plot(sent_gryffindor_vs_all[0][1], c="red", alpha=1, linewidth=2)
axs[1].plot(sent_gryffindor_vs_all[1][1], c="black", alpha=1, linewidth=2)
axs[1].axhline(y = 0.05, color ="black", linestyle = '-', label="Neutral", alpha=0.5)
axs[1].axhline(y = -0.05, color ="black", linestyle = '-', alpha=0.5)
axs[1].set_xlim(0, len(sent_ravenclaw_vs_all[0][1]))
axs[1].set_ylabel("Average sentiment", fontsize=16)
axs[1].tick_params(axis='y', which='major', labelsize=16)
axs[1].set_xticks([])

axs[2].plot(sent_ravenclaw_vs_all[0][1], c="blue", alpha=1, linewidth=2)
axs[2].plot(sent_ravenclaw_vs_all[1][1], c="black", alpha=1, linewidth=2)
axs[2].axhline(y = 0.05, color ="black", linestyle = '-', label="Neutral", alpha=0.5)
axs[2].axhline(y = -0.05, color ="black", linestyle = '-', alpha=0.5)
axs[2].set_xlim(0, len(sent_ravenclaw_vs_all[0][1]))
axs[2].set_ylabel("Average sentiment", fontsize=16)
axs[2].tick_params(axis='y', which='major', labelsize=16)
axs[2].set_xticks([])

axs[3].plot(sent_hufflepuff_vs_all[0][1], c="orange", alpha=1, linewidth=2)
axs[3].plot(sent_hufflepuff_vs_all[1][1], c="black", alpha=1, linewidth=2)
axs[3].axhline(y = 0.05, color ="black", linestyle = '-', label="Neutral", alpha=0.5)
axs[3].axhline(y = -0.05, color ="black", linestyle = '-', alpha=0.5)
axs[3].set_xlim(0, len(sent_ravenclaw_vs_all[0][1]))
axs[3].set_ylabel("Average sentiment", fontsize=16)
axs[3].tick_params(axis='y', which='major', labelsize=16)
xs = []
labels = []
for book, label, vertical_line in book_list_wchapter:
    axs[0].axvline(x = vertical_line, color = 'black')
    axs[1].axvline(x = vertical_line, color = 'black')
    axs[2].axvline(x = vertical_line, color = 'black')
    axs[3].axvline(x = vertical_line, color = 'black')
    xs.append(vertical_line)
    labels.append(label)
    
    
plt.sca(axs[3])
plt.xticks(xs, labels ,rotation=90, fontsize=16)

plt.savefig("sentimenthouses.png", transparent=True)

In [ ]:
# Average over the entire text for each house

def house_average_total(character_names, book_list):
    # Count number of chapters
    num_chapters = 0
    for book in book_list:
        for chapter in os.listdir(book):
            if "replaced" in chapter:
                num_chapters += 1
    # Counter for current chapter
    c = 0
    slytherin_occ = 0
    rest_occ = 0
    slytherin_sent = 0
    rest_sent = 0

    slytherins_sents = []
    rest_sents = []

    slytherin_total_sent = 0
    slytherin_total_occ = 0
    rest_total_sent = 0
    rest_total_occ = 0

    # For each member of the group
    for member in character_names:
        # For each book in the list
        rest_occ = 0
        rest_sent = 0
        for book in book_list:  
            # For each chapter going by numbering
            for chapter in os.listdir(book):
                if "replaced" in chapter:
                    # Init counter for counting occurences in chapter for average
                    occurences = 0

                    # Read in the chapter
                    with open(book + chapter) as f:
                        raw = f.read()
                    tokens = nltk.word_tokenize(raw)
                    # Prepare nltk text
                    text = nltk.Text(tokens)
                    # Make concordance for that member for that chapter
                    con_list = text.concordance_list(member, width = 3 * average_length_of_sentence)

                    # For each concordance line in the list
                    for con in con_list:
                        # Calculate the sentiment
                        sent = con_sentiment(con)
                        if sent != 0:
                            # Sum up the sentiment for that chapter for that member
                            # with sentiments for all other members of group
                            #print(f"c = {c}")
                            rest_sent += sent
                            rest_occ += 1
        if rest_occ != 0:
            rest_total_sent += rest_sent
            rest_total_occ += rest_occ
    return rest_total_sent / rest_total_occ

In [ ]:
house_average_total(slytherins, book_list)

In [ ]:
house_average_total(gryffindors, book_list)

In [ ]:
house_average_total(ravenclaws, book_list)

In [ ]:
house_average_total(hufflepuffs, book_list)

## Sentiment for communities

In [ ]:
book1_community0 = ['Albus_Dumbledore', 'Minerva_McGonagall', 'Harry_Potter', 'Tom_Riddle', 'Rubeus_Hagrid', 'Sirius_Black', 'Dedalus_Diggle', 'Troy', 'Seamus_Finnigan', 'Quirinus_Quirrell', 'Hermione_Granger', 'Neville_Longbottom', 'Charles_Weasley', 'Ronald_Weasley', 'Gringotts_Head_Goblin', 'Bloody_Baron', 'Fat_Lady', 'Peeves', 'Parvati_Patil', 'Dean_Thomas', 'Fluffy', 'Knight_with_an_arrow_sticking_out_of_his_forehead', 'Irma_Pince', 'Severus_Snape', 'Miles_Bletchley', 'Edward_Lupin', 'Hedwig', 'Norberta', 'Fang', 'Harper', 'Trevor', 'Garrick_Ollivander', 'Griphook', 'Cornelius_Fudge', 'Doris_Crockford', 'Malkin', 'Justin_Finch-Fletchley', 'Lisa_Turpin', 'Pansy_Parkinson']
book1_community1 = ['Jack_Sloper', 'Mundungus_Fletcher', 'Goyle', 'Percy_Weasley', 'Bloody_Baron', 'Peeves', 'Oliver_Wood', 'Crabbe', 'Fred_Weasley', 'George_Weasley', 'Adrian_Pucey', 'Alicia_Spinnet', 'Angelina_Johnson', 'Terence_Higgs', 'Rolanda_Hooch', 'Marcus_Flint', 'Lee_Jordan', 'Katie_Bell', 'John_Dawlish', 'Draco_Malfoy', 'Edward_Lupin', 'Ginevra_Weasley', 'Rose_Granger-Weasley', 'Morag_MacDougal', 'Blaise_Zabini', 'Nott', 'Argus_Filch', 'Gregory_Goyle']

In [ ]:
avgsent_book1_c0 = avgsent_group(book1_community0, "Average for main characters", ["B1/"], 3)
avgsent_book1_c1 = avgsent_group(book1_community1, "Average for main characters", ["B1/"], 3)

In [ ]:
book1_plot = [avgsent_book1_c0] + [avgsent_book1_c1]

In [ ]:
plot_sentiments(book1_plot, "This is just a text", [], False)

In [ ]:
book2_community0 = ['Harry_Potter', 'Rubeus_Hagrid', 'Hermione_Granger', 'Ronald_Weasley', 'Rita_Skeeter', 'Albus_Dumbledore', 'Minerva_McGonagall', 'Colin_Creevey', 'Myrtle_Warren', 'Fang', 'Irma_Pince', 'Crabbe', 'Millicent_Bulstrode', 'Seamus_Finnigan', 'Gilderoy_Lockhart', 'Dean_Thomas', 'Goyle', 'Neville_Longbottom', 'Mrs_Norris', 'Fawcett', 'Cook_of_The_Hanged_Man', 'Fawkes', 'Fluffy', 'Cornelius_Fudge', 'Aragog', 'Miles_Bletchley', 'Frank_Longbottom', 'Basil', 'Armando_Dippet', 'Errol', 'Severus_Snape', 'Fat_Lady', 'Tufty', 'Argus_Filch', 'Edward_Lupin', 'Lavender_Brown', 'Parvati_Patil']
book2_community1 = ['Draco_Malfoy', 'Arthur_Weasley', 'Fred_Weasley', 'George_Weasley', 'Adrian_Pucey', 'Alicia_Spinnet', 'Angelina_Johnson', 'Rolanda_Hooch', 'Oliver_Wood', 'Marcus_Flint', 'Lee_Jordan', 'Lucius_Malfoy', 'Ginevra_Weasley', 'Molly_Weasley', 'Percy_Weasley', 'William_Weasley', 'Peter_Pettigrew', 'Gringotts_Head_Goblin', 'Charles_Weasley', 'Penelope_Clearwater', 'Hermes', 'Perkins', "Hermione_Granger's_father", "Hermione_Granger's_mother", 'Alice_Longbottom', 'Borgin', 'Katie_Bell']
avgsent_book2_c0 = avgsent_group(book2_community0, "Average for main characters", ["B2/"], 3)
avgsent_book2_c1 = avgsent_group(book2_community1, "Average for main characters", ["B2/"], 3)
book2_plot = [avgsent_book2_c0] + [avgsent_book2_c1]
plot_sentiments(book2_plot, "This is just a text", [], False)

In [ ]:
book3_community0 = ['Minerva_McGonagall', 'Harry_Potter', 'Rubeus_Hagrid', 'Hermione_Granger', 'Ronald_Weasley', 'Bathilda_Bagshot', 'Albus_Dumbledore', 'Dot', 'Cornelius_Fudge', 'Peeves', 'Seamus_Finnigan', 'Severus_Snape', 'Sybill_Trelawney', 'Neville_Longbottom', 'Lucius_Malfoy', 'Norberta', 'Buckbeak', 'Crookshanks', 'Derek', 'Fang', 'Cadogan', 'Chambers', 'Dean_Thomas', 'Fluffy', 'Eric_Munch', 'Alice_Longbottom', 'Fat_Lady', 'Lavender_Brown', 'Charles_Weasley', 'Parvati_Patil', 'Bane', 'Poppy_Pomfrey', 'Walden_Macnair', 'Nott', 'Wailing_Widow', 'Irma_Pince', 'Colin_Creevey', 'Frank_Longbottom', 'Igor_Karkaroff']
book3_community3 = ['Tom_Riddle', 'Lily_J._Potter', 'James_Potter_I', 'Remus_Lupin', 'Sirius_Black', 'Nicholas_de_Mimsy-Porpington', 'Peter_Pettigrew', 'Rosmerta', 'Troy', 'Hannah_Abbott']
avgsent_book3_c0 = avgsent_group(book3_community0, "Average for main characters", ["B3/"], 3)
avgsent_book3_c3 = avgsent_group(book3_community3, "Average for main characters", ["B3/"], 3)
book3_plot = [avgsent_book3_c0] + [avgsent_book3_c3]
plot_sentiments(book3_plot, "This is just a text", [], False)

In [ ]:
book4_community0 = ['Harry_Potter', 'Tom_Riddle', 'Frank_Bryce', 'Dot', 'Bertha_Jorkins', 'Peter_Pettigrew', 'Nagini', 'Igor_Karkaroff', 'Albus_Dumbledore', 'Mundungus_Fletcher', 'Rubeus_Hagrid', 'Hermione_Granger', 'Alastor_Moody', 'Ronald_Weasley', 'Pigwidgeon', 'Cornelius_Fudge', 'Crookshanks', 'Viktor_Krum', 'Rita_Skeeter', 'Minerva_McGonagall', 'Sirius_Black', 'Aurora_Sinistra', 'Bloody_Baron', 'Fat_Friar', 'Nicholas_de_Mimsy-Porpington', 'Fang', 'Fat_Lady', 'Stewart_Ackerley', 'Peeves', 'Sybill_Trelawney', 'Winky', 'Blast-Ended_Skrewt', 'Hedwig', 'Norberta', 'Septima_Vector', 'Eloise_Midgen', 'Lily_J._Potter', 'James_Potter_I', 'Buckbeak', 'Cedric_Diggory', 'Fleur_Delacour', 'Bartemius_Crouch_Senior', 'Dobby', 'Cho_Chang', 'Poliakoff', 'Severus_Snape', 'Alice_Longbottom', 'Garrick_Ollivander', 'Justin_Finch-Fletchley', 'Pansy_Parkinson', 'Luna_Lovegood', 'Rosmerta', 'Trevor', 'Wailing_Widow', 'Fluffy', 'Myrtle_Warren', 'Stebbins_(1990s_Hogwarts_student)', 'Roger_Davies', 'Bozo', 'Wilhelmina_Grubbly-Plank', 'Mrs_Norris', 'Irma_Pince', 'Errol', 'Fawkes', 'Lisa_Turpin', 'Harper', 'Olympe_Maxime', 'Hermes', 'Salazar_Slytherin', 'Stanley_Shunpike', 'Remus_Lupin', 'Poppy_Pomfrey']
book4_community3 = ['Goyle', 'Draco_Malfoy', 'Crabbe', 'Nott', 'Aberforth_Dumbledore', 'Vincent_Crabbe', 'Avery_II', 'Avery_I', 'Rose_Granger-Weasley', 'Lestrange', 'Bellatrix_Lestrange', 'Rabastan_Lestrange', 'Rodolphus_Lestrange', 'Lucius_Malfoy', 'Narcissa_Malfoy']
avgsent_book4_c0 = avgsent_group(book4_community0, "Average for main characters", ["B3/"], 3)
avgsent_book4_c3 = avgsent_group(book4_community3, "Average for main characters", ["B3/"], 3)
book4_plot = [avgsent_book4_c0] + [avgsent_book4_c3]
plot_sentiments(book4_plot, "This is just a text", [], False)

In [ ]:
book5_community0 = ['Albus_Dumbledore', 'Harry_Potter', 'Rubeus_Hagrid', 'Hermione_Granger', 'Cook_of_The_Hanged_Man', 'Ronald_Weasley', 'Hedwig', 'Cho_Chang', 'Cornelius_Fudge', 'Wilhelmina_Grubbly-Plank', 'Rita_Skeeter', 'Minerva_McGonagall', 'Tom_Riddle', 'Bloody_Baron', 'Nicholas_de_Mimsy-Porpington', 'Cedric_Diggory', 'Euan_Abercrombie', 'Fat_Lady', 'Rose_Zeller', 'Sybill_Trelawney', 'Dolores_Umbridge', 'Aurora_Sinistra', 'Knight_with_an_arrow_sticking_out_of_his_forehead', 'Cadogan', 'Peeves', 'Quirinus_Quirrell', 'Viktor_Krum', 'Winky', 'Hermes', 'Eric_Munch', 'Malkin', 'Igor_Karkaroff', 'Basil', 'Miles_Bletchley', 'Fawkes', 'Griselda_Marchbanks', 'Tiberius_Ogden', 'Zacharias_Smith', 'Eloise_Midgen', 'Dobby', 'Marietta_Edgecombe', 'Olympe_Maxime', 'Fang', 'Dilys_Derwent', 'Everard', 'Harper', 'Agnes', 'Stanley_Shunpike', 'Lisa_Turpin', 'Septima_Vector', 'Roger_Davies', 'Puddifoot', 'Firenze', 'Bane', 'Magorian', 'Norberta', 'Grawp', 'Aragog', 'Chambers', 'Eddie_Carmichael', 'Tofty', 'Clara_Ivanova', 'Ronan', 'Dot', 'Williamson']
book5_community1 = ['Nymphadora_Tonks', 'Sirius_Black', 'Alastor_Moody', 'Kingsley_Shacklebolt', 'Arthur_Weasley', 'Molly_Weasley', 'Kreacher', 'Doris_Crockford', 'Sturgis_Podmore', 'Alice_Longbottom', 'William_Weasley', 'Gloomy_nuns', 'Gilderoy_Lockhart', 'Remus_Lupin', 'Donaghan_Tremlett', 'Lucius_Malfoy', 'James_Potter_I', 'Myron_Wagtail', 'Wailing_Widow', 'Perkins', 'Buckbeak', 'Jack_Sloper', 'Phineas_Nigellus_Black', 'Augustus_Rookwood', 'Armando_Dippet', 'Urquhart_Rackharrow', 'Hippocrates_Smethwyck', 'Lestrange', 'Bellatrix_Lestrange', 'Broderick_Bode', 'Frank_Longbottom', 'Antonin_Dolohov', 'Gideon_Crumb', 'Miriam_Strout', 'Avery_II', 'Avery_I', 'Nott', 'Lily_J._Potter', 'Stebbins_(1990s_Hogwarts_student)', 'Peter_Pettigrew', 'Aberforth_Dumbledore', 'Rabastan_Lestrange', 'Rodolphus_Lestrange', 'Mulciber_II', 'Jugson', 'Severus_Snape', 'Narcissa_Malfoy', 'Edward_Tonks', 'Bartemius_Crouch_Senior', 'Euphemia_Potter', 'Fleamont_Potter', 'Andromeda_Tonks', 'Regulus_Black', 'Hestia_Jones', 'Amelia_Bones', 'Elphias_Doge', 'Dedalus_Diggle', 'Edgar_Bones', 'Emmeline_Vance']
avgsent_book5_c0 = avgsent_group(book5_community0, "Average for main characters", ["B3/"], 3)
avgsent_book5_c1 = avgsent_group(book5_community1, "Average for main characters", ["B3/"], 3)
book5_plot = [avgsent_book5_c0] + [avgsent_book5_c1]
plot_sentiments(book5_plot, "This is just a text", [], False)

In [ ]:
book6_community0 = ['Albus_Dumbledore', 'Harry_Potter', 'Tom_Riddle', 'Sirius_Black', 'Gloomy_nuns', 'Amelia_Bones', 'Cornelius_Fudge', 'Emmeline_Vance', 'Rufus_Scrimgeour', 'Sybill_Trelawney', 'Cecilia', 'Marvolo_Gaunt', 'Morfin_Gaunt', 'Merope_Riddle', 'Salazar_Slytherin', 'Bloody_Baron', 'Miles_Bletchley', 'Bob_Ogden', 'Fawkes', 'Rita_Skeeter', 'Lisa_Turpin', 'Igor_Karkaroff', 'Edward_Lupin', 'Minerva_McGonagall', 'Rubeus_Hagrid', 'Vernon_Dursley', 'Frank_Longbottom', 'Hedwig', 'Grawp', 'Aragog', 'Buckbeak', 'Eloise_Midgen', 'Fang', 'Wilhelmina_Grubbly-Plank', 'Stanley_Shunpike', 'Rosmerta', 'Irma_Pince', 'Rose_Granger-Weasley', 'Cole', 'Dennis', 'Phineas_Nigellus_Black', 'Eric_Munch', 'Nott', 'Winky', 'Firenze', 'Dolores_Umbridge', 'Bartemius_Crouch_Senior', 'James_Potter_I', 'Troy', 'Lily_J._Potter', 'Hokey', 'Armando_Dippet', 'Mulciber_II', 'Pomona_Sprout', 'Cedric_Diggory', 'Nagini', 'Graham_Montague', 'Fluffy', 'Regulus_Black', 'Filius_Flitwick', 'Everard', 'Dudley_Dursley', 'Petunia_Dursley', 'Augusta_Longbottom', 'Ambrosius_Flume', 'Alice_Longbottom', 'Tufty']
book6_community1 = ['Hermione_Granger', 'Goyle', 'Ronald_Weasley', 'Hannah_Abbott', 'Lavender_Brown', 'Ginevra_Weasley', 'Pigwidgeon', 'Arnold', 'Demelza_Robins', 'Romilda_Vane', 'Parvati_Patil', 'Borgin', 'Katie_Bell', 'Crabbe', 'Seamus_Finnigan', 'Mundungus_Fletcher', 'Dean_Thomas', 'Nicholas_de_Mimsy-Porpington', 'Wailing_Widow', 'Fat_Lady', 'Puddifoot', 'Leanne', 'Muriel', 'Blast-Ended_Skrewt', 'Mrs_Norris', 'Cho_Chang', 'Crookshanks', 'Viktor_Krum', 'Vaisey', 'Myrtle_Warren', 'Susan_Bones', 'Ernest_Macmillan', 'Wilkie_Twycross', 'Gregory_Goyle', 'Pansy_Parkinson', 'Dot', 'Septima_Vector', 'Emma_Dobbs', 'Michael_Corner', 'Vincent_Crabbe', 'Thestral']
book6_community2 = ['Kingsley_Shacklebolt', 'Lee_Jordan', 'Jack_Sloper', 'Cook_of_The_Hanged_Man', 'Arthur_Weasley', 'Fred_Weasley', 'George_Weasley', 'Knight_with_an_arrow_sticking_out_of_his_forehead', 'Ritchie_Coote', 'Jimmy_Peakes', 'Nymphadora_Tonks', 'Molly_Weasley', 'Ernest_Prang', 'Fleur_Delacour', 'William_Weasley', 'Donaghan_Tremlett', 'Eldred_Worple', 'Sanguini', 'Myron_Wagtail', 'Remus_Lupin', 'Charles_Weasley', 'Percy_Weasley', 'Gringotts_Head_Goblin', 'Dirk_Cresswell', 'Gabrielle_Delacour', 'Alastor_Moody', 'Garrick_Ollivander', 'Florean_Fortescue', 'Malkin', 'Arabella_Figg']
book6_community4 = ['Severus_Snape', 'Lestrange', 'Bellatrix_Lestrange', 'Draco_Malfoy', 'Fenrir_Greyback', 'Avery_I', 'Lucius_Malfoy', 'Peter_Pettigrew', 'Eileen_Prince', 'Alecto_Carrow', 'Amycus_Carrow', 'Quirinus_Quirrell', 'Narcissa_Malfoy', 'Corban_Yaxley', 'Tobias_Snape']
avgsent_book6_c0 = avgsent_group(book6_community0, "Average for main characters", ["B6/"], 3)
avgsent_book6_c1 = avgsent_group(book6_community1, "Average for main characters", ["B6/"], 3)
avgsent_book6_c2 = avgsent_group(book6_community2, "Slytherin for main characters", ["B6/"], 3)
avgsent_book6_c4 = avgsent_group(book6_community4, "Gryffindor for main characters", ["B6/"], 3)
book6_plot = [avgsent_book6_c2] + [avgsent_book6_c4]
plot_sentiments(book6_plot, "This is just a text", [], False)